# Practical Exam: Hotel Operations

LuxurStay Hotels is a major, international chain of hotels. They offer hotels for both business and leisure travellers in major cities across the world. The chain prides themselves on the level of customer service that they offer. 

However, the management has been receiving complaints about slow room service in some hotel branches. As these complaints are impacting the customer satisfaction rates, it has become a serious issue. Recent data shows that customer satisfaction has dropped from the 4.5 rating that they expect. 

You are working with the Head of Operations to identify possible causes and hotel branches with the worst problems. 

## Data

The following schema diagram shows the tables available. You have only been provided with data where customers provided a feedback rating.

![hotel_operations](hotel_operations.png)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `branch` table, and the data team have provided the following data description. 

Write a query to return data matching this description, including identifying and cleaning all invalid values. You must match all column names and description criteria. Your output should be a DataFrame named 'clean_branch_data'.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|id | Nominal. The unique identifier of the hotel. </br>Missing values are not possible due to the database structure.|
| location | Nominal. The location of the particular hotel. One of four possible values, 'EMEA', 'NA', 'LATAM' and 'APAC'. </br>Missing values should be replaced with “Unknown”. |
| total_rooms | Discrete. The total number of rooms in the hotel. Must be a positive integer between 1 and 400. </br>Missing values should be replaced with the default number of rooms, 100. |
| staff_count | Discrete. The number of staff employeed in the hotel service department. </br>Missing values should be replaced with the total_rooms multiplied by 1.5. |
| opening_date | Discrete. The year in which the hotel opened. This can be any value between 2000 and 2023. </br>Missing values should be replaced with 2023. |
| target_guests | Nominal. The primary type of guest that is expected to use the hotel. Can be one of 'Leisure' or 'Business'. </br>Missing values should be replaced with 'Leisure'. |

In [5]:
SELECT
    id,

    -- Clean location
    CASE
        WHEN location IN ('EMEA', 'NA', 'LATAM', 'APAC') THEN location
        ELSE 'Unknown'
    END AS location,

    -- Clean total_rooms
    CASE
        WHEN total_rooms::text ~ '^\d+$' AND total_rooms::int BETWEEN 1 AND 400
        THEN total_rooms::int
        ELSE 100
    END AS total_rooms,

    -- Clean staff_count
    CASE
        WHEN staff_count::text ~ '^\d+$' THEN staff_count::int
        ELSE 
            (CASE
                WHEN total_rooms::text ~ '^\d+$' AND total_rooms::int BETWEEN 1 AND 400
                THEN total_rooms::int
                ELSE 100
            END) * 1.5
    END AS staff_count,

    -- Clean opening_date
    CASE
        WHEN opening_date::text ~ '^\d+$' AND opening_date::int BETWEEN 2000 AND 2023
        THEN opening_date::int
        ELSE 2023
    END AS opening_date,

    -- Clean target_guests
    CASE
        WHEN target_guests IN ('Leisure', 'Business') THEN target_guests
        ELSE 'Leisure'
    END AS target_guests

FROM branch;



,id,location,total_rooms,staff_count,opening_date,target_guests
0,1,LATAM,168,178,2017,Business
1,2,APAC,154,82,2010,Leisure
2,3,APAC,212,467,2003,Leisure
3,4,APAC,230,387,2023,Business
4,5,APAC,292,293,2002,Business
...,...,...,...,...,...,...
95,96,APAC,237,257,2000,Business
96,97,APAC,107,169,2005,Business
97,98,EMEA,196,126,2002,Leisure
98,99,APAC,242,251,2021,Business


# Task 2

The Head of Operations wants to know whether there is a difference in time taken to respond to a customer request in each hotel. They already know that different services take different lengths of time. 

Calculate the average and maximum duration for each branch and service. 
- Your output should be a DataFrame named 'average_time_service'
- It should include the columns `service_id`, `branch_id`, `avg_time_taken` and `max_time_taken`
- Values should be rounded to two decimal places where appropriate. 

In [6]:
-- Write your query for task 2 in this cell
SELECT
    service_id,
    branch_id,
    ROUND(AVG(time_taken)::numeric, 2) AS avg_time_taken,
    ROUND(MAX(time_taken)::numeric, 2) AS max_time_taken
FROM Request
GROUP BY branch_id, service_id
ORDER BY branch_id, service_id;



,service_id,branch_id,avg_time_taken,max_time_taken
0,1,1,2.44,12.0
1,2,1,12.73,15.0
2,3,1,7.00,7.0
3,1,2,2.14,6.0
4,2,2,13.46,15.0
...,...,...,...,...
380,3,99,6.97,8.0
381,4,99,9.13,13.0
382,2,100,14.00,14.0
383,3,100,6.89,8.0


# Task 3

The management team want to target improvements in `Meal` and `Laundry` service in Europe (`EMEA`) and Latin America (`LATAM`). 

Write a query to return the `description` of the service, the `id` and `location` of the branch, the id of the request as `request_id` and the `rating` for the services and locations of interest to the management team. 

Your output should be a DataFrame named 'target_hotels'.

Use the original branch table, not the output of task 1. 

In [7]:
-- Write your query for task 3 in this cell
SELECT
    s.description AS service_description,
    b.id AS branch_id,
    b.location,
    r.id AS request_id,
    r.rating
FROM Request r
JOIN Branch b
    ON r.branch_id = b.id
JOIN Service s
    ON r.service_id = s.id
WHERE s.description IN ('Meal', 'Laundry')
  AND b.location IN ('EMEA', 'LATAM')
ORDER BY b.location, b.id, s.description;


,service_description,branch_id,location,request_id,rating
0,Laundry,7,EMEA,12290,4
1,Laundry,7,EMEA,2462,3
2,Laundry,7,EMEA,5870,3
3,Laundry,7,EMEA,2921,4
4,Laundry,7,EMEA,11967,4
...,...,...,...,...,...
5042,Laundry,93,LATAM,12159,4
5043,Meal,93,LATAM,5230,4
5044,Meal,93,LATAM,14942,4
5045,Meal,93,LATAM,14947,3


# Task 4

So that you can take a more detailed look at the lowest performing hotels, you want to get service and branch information where the average rating for the branch and service combination is lower than 4.5 - the target set by management.  

- Your output should be a DataFrame named 'average_rating'
- It should return the `service_id` and `branch_id`, and the average rating (`avg_rating`)
- Values should be rounded to 2 decimal places where appropriate.

In [8]:
-- Write your query for task 4 in this cell
SELECT
    service_id,
    branch_id,
    ROUND(AVG(rating)::numeric, 2) AS avg_rating
FROM Request
GROUP BY branch_id, service_id
HAVING AVG(rating) < 4.5
ORDER BY branch_id, service_id;


,service_id,branch_id,avg_rating
0,1,1,3.66
1,2,1,3.91
2,3,1,4.00
3,3,3,4.00
4,1,4,3.75
...,...,...,...
210,3,99,3.93
211,4,99,3.83
212,2,100,3.00
213,3,100,3.89
